## S3 upload and download

In [1]:
import os
from brtdevkit.util.aws import s3

Internal AWS credentials will be removed from future brt-devkit versions.


In [2]:
s3_client = s3.S3()
BRT_DEVKIT_WRITE_S3_BUCKET = 'mesa-states'
MODEL_TRAINING_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/model_training"
PACK_PERCEPTION_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/pack_perception"
bucket_name=BRT_DEVKIT_WRITE_S3_BUCKET

In [ ]:
# upload file
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/Jupiter_train_v4_53_missing_human_relabeled.tar"
key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
file_path = "/home/bluerivertech/li.yu/data/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
s3_client.upload_file(
    bucket_name=bucket_name, key=key, file_path=file_path)
# Jupiter_2022_fn_bag1.tar
# Jupiter_2022Jan15Feb15_rock_stops.tar
# Jupiter_train_v4_53_missing_human_relabeled.tar
# Jupiter_train_v4_53_heavy_dust_relabeled.tar

In [4]:
# download file
# run_id = '45_3_6class_ml_pp_seg_lite_cloud_v10_rotate_fix'
# name = 'driveable_terrain_model_val_bestmodel.pth'
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/{run_id}/{name}"

key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"

target_path = '/data/jupiter/li.yu/data/Jupiter_train_v4_53/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar'
s3_client.download_file(bucket_name, key, target_path)

True

## Resave pretrained weights

In [1]:
import torch
from collections import OrderedDict

# d = torch.load('/mnt/sandbox1/ben.cline/data/20240110/fpn_80k_nextvit_small_1n1k6m_pretrained.pth', map_location=lambda storage, loc: storage)
# print(d.keys())
# ori_keys = d['state_dict'].keys()

In [2]:
# supervised pretraining lite12 model
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = 'openimages_v7_brtlite12_0211'
checkpoint = 'checkpoint_best'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict['model'].items():
    output_dict[k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())
torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])


In [3]:
# supervised pretraining
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = 'openimages_v7_nextvit_small_structuralreparam_0311'
checkpoint = 'checkpoint'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict['model'].items():
    output_dict['backbone.' + k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())

# check difference between open source weights and OpenImages pretrained weights
# print(set(output_dict['state_dict'].keys()) - set(ori_keys))

torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])


In [2]:
# self-supervised pretraining
checkpoint_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
# model = 'brt1m_spark_nextvit_small_0307'
model = 'openimages_v7_spark_nextvit_small_0226'
checkpoint = 'nextvit_small_pretrained_timm_style_upto289epochs'  # checkpoint or checkpoint_best
input_dict = torch.load(os.path.join(checkpoint_dir, model, f'{checkpoint}.pth'), map_location=lambda storage, loc: storage)
output_dict = OrderedDict()
for k, v in input_dict.items():
    output_dict['backbone.' + k] = v
output_dict = {'state_dict': output_dict}
print(output_dict.keys())

# # check difference between open source weights and OpenImages pretrained weights
# print(set(output_dict['state_dict'].keys()) - set(ori_keys))

torch.save(output_dict, os.path.join(checkpoint_dir, model, f'{checkpoint}_brt_compatible.pth'))

dict_keys(['state_dict'])
